In [16]:
import numpy as np
import scipy.sparse

train = scipy.sparse.load_npz("./data/movielens.adjacency.train.npz")
test = scipy.sparse.load_npz("./data/movielens.adjacency.test.npz")

In [63]:
import implicit

model = implicit.als.AlternatingLeastSquares(factors=100, regularization=1.0, iterations=15)
model.fit(train.astype(float).T.tocsr())

100%|██████████| 15.0/15 [00:02<00:00,  5.36it/s]


In [60]:
import itertools

user_ids, movie_ids = test.nonzero()
ranks = []
i = 0
for user_id, movie_id in list(zip(user_ids,movie_ids)):
    #ranking = model.recommend(user_id, train.astype(float), N=test.shape[1], )
    #ranking = list(map(lambda x: x[0], ranking))
    #rank = 1 - ranking.index(movie_id) / test.shape[1]
    #ranks.append(rank)
    
    if i % 1000 == 0 and i > 0:
        print(i, np.mean(ranks))
        
    i += 1
    
    N = test.shape[1]
    user = model._user_factor(user_id, train.astype(float), False)
    scores = model.item_factors.dot(user)
    best = sorted(enumerate(scores), key=lambda x: -x[1])
    ranking = list(itertools.islice((rec[0] for rec in best), N))
    rank = 1 - ranking.index(movie_id) / N
    ranks.append(rank)

1000 0.9048054149797572


KeyboardInterrupt: 

In [64]:
from implicit.evaluation import precision_at_k, mean_average_precision_at_k

pr = precision_at_k(model, train, test, K=10, show_progress=True, num_threads=1)
mean_pr = mean_average_precision_at_k(model, train, test, K=10, show_progress=True, num_threads=1)
print(pr, mean_pr)

100%|██████████| 6040/6040 [00:02<00:00, 2254.90it/s]

0.2324503311258278 0.09831224377168095


In [28]:
# ALS, reg=0.1
# 1000 0.9183556443556443
# 2000 0.9167468265867067
# 3000 0.9174039986671109
# 4000 0.9171358160459885
# 5000 0.9168736652669467

# ALS, reg=0.01


310

In [39]:
from scipy.stats import percentileofscore

users = model.user_factors
movies = model.item_factors

user_ids, movie_ids = test.nonzero()
ranks = []
for idx in range(test.shape[0]):
    if idx % 10000 == 0 and idx > 0:
        print(idx, np.mean(ranks))
        
    user_id = user_ids[idx]
    movie_id = movie_ids[idx]
    sims = movies.dot(users[user_id,:])
    rank = percentileofscore(sims, sims[movie_id])
    ranks.append(rank)
    
np.mean(ranks)

90.10588569455987

In [ ]:
### Netflix, ALS, reg=0.01 
# 10000 92.05844850872256
# 20000 92.59294400675296
# 30000 92.82760195085349
# 40000 92.72613512943163
# 50000 92.94068761958357
# 60000 93.16114584505719
# 70000 93.35329954176379
# 80000 93.50621324563872
# 90000 93.67388079159632
# 100000 93.78302613956104
# 110000 93.86933475725175
# 120000 93.94542836709812
# 130000 94.00160951906844
# 140000 94.06216896454697
# 150000 94.11513489026447
# 160000 94.14663637802474
# 170000 94.03039249892414
# 180000 93.94640877258799
# 190000 93.88283739596598
# 200000 93.84967079347214
# 210000 93.81465484899643
# 220000 93.81932883562695
# 230000 93.83166911746716
# 240000 93.88920907193771
# 250000 93.90744452447946
# 260000 93.93891532834076
# 270000 93.95845405281476
# 280000 93.95183502492159
# 290000 93.91815445442725
# 300000 93.90756973363345
# 310000 93.89090062083612
# 320000 93.89187666185985
# 330000 93.9174569413891
# 340000 93.9018869624946
# 350000 93.8774897017445
# 360000 93.85232795598073
# 370000 93.85002647949018
# 380000 93.83749853389804
# 390000 93.81050684674544
# 400000 93.7933132667417
# 410000 93.75588784879972
# 420000 93.77390031353005
# 430000 93.7793130570206
# 440000 93.76456006036733
# 450000 93.74721596948666
# 460000 93.72798154559469
# 470000 93.68952901136271

In [ ]:
### Movielens, ALS, reg=0.01
# 89.96889830281255